<img align="right" src="images/etcbc.png"/>
<img align="right" src="images/peshitta_small.png"/>
<img align="right" src="images/tf-small.png"/>


# Link Syriaca data

This notebook links Syriaca data to the Syriac New Testament.

We add the links as features to the
[Text-Fabric](https://github.com/Dans-labs/text-fabric) 
representation of SyrNT.

In [39]:
import sys, os, collections
from IPython.display import display, Markdown
from tf.fabric import Fabric

In [6]:
REPO = '~/github/etcbc/linksyr'
SOURCE = 'syrnt'
CORPUS = f'{REPO}/data/tf/{SOURCE}'

In [2]:
TF = Fabric(locations=[CORPUS], modules=[''], silent=False )

This is Text-Fabric 3.2.2
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

37 features found and 0 ignored


# Load Features
We load all available features of the SyrNT data.

In [3]:
api = TF.load('', silent=True)
allFeatures = TF.explore(silent=True, show=True)
loadableFeatures = allFeatures['nodes'] + allFeatures['edges']
TF.load(loadableFeatures, add=True, silent=False)
api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.04s B aspect               from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.04s B demonstrative_category from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.02s B feminine_he_dot      from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.03s B gender               from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.04s B grammatical_category from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.04s B noun_type            from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.03s B number               from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.04s B numeral_type         from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.04s B participle_type      from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.03s B person               from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.03s B prefix               from /Users/dirk/github/etcbc/l

## Syriaca data

We load the index of people and places.

In [44]:
SYRIACA = os.path.expanduser(f'{REPO}/data/syriaca')
SC_PEOPLE = f'{SYRIACA}/index_of_persons.csv'
SC_PLACES = f'{SYRIACA}/index_of_places.csv'

SC_URL = 'http://syriaca.org'
SC_PLACE = 'place'
SC_PERSON = 'person'

SC_CONFIG = (
    (SC_PERSON, SC_URL, SC_PEOPLE),
    (SC_PLACE, SC_URL, SC_PLACES),    
)

SC_TYPES = tuple(x[0] for x in SC_CONFIG)

SC_FIELDS = ('trans', 'syriac', 'id')

NA_SYRIAC = {
    '[Syriac Not Available]', 
    '[Syriac Not', 
    '[Syriac',
}

In [53]:
tables = {}
irregular = {}

(transF, syriacF, idF) = SC_FIELDS

for (dataType, baseUrl, dataFile) in SC_CONFIG:
    tables[dataType] = {field: {} for field in SC_FIELDS}
    irregular[dataType] = set()
    dest = tables[dataType]
    irreg = irregular[dataType]
    table = dest[idF]
    indexTrans = dest[transF]
    indexSyriac = dest[syriacF]
    with open(dataFile) as fh:
        for (i, line) in enumerate(fh):
            (transV, syriacV, idV) = line.rstrip('\n').split('\t')
            prefix = f'{baseUrl}/{dataType}/'
            if idV.startswith(prefix):
                idV = idV.replace(prefix, '', 1)
            else:
                irreg.add(idV)
            table[idV] = (transV, syriacV)
            indexTrans.setdefault(transV, set()).add(idV)
            if syriacV not in NA_SYRIAC:
                if '[' in syriacV:
                    print(f'WARNING {dataType} line {i+1}: syriac value "{syriacV}"')
                indexSyriac.setdefault(syriacV, set()).add(idV)

In [55]:
for (dataType, data) in tables.items():
    table = data[idF]
    irreg = irregular[dataType]
    print(f'''
{dataType:>12}s: {len(table):>5} (irregular: {len(irreg):>4})
{"by syriac":>12} : {len(data[syriacF]):>5}
{"by trans":>12} : {len(data[transF]):>5}
''')


      persons:  2371 (irregular:    0)
   by syriac :  1503
    by trans :  1964


       places:  2488 (irregular:    0)
   by syriac :   527
    by trans :  2165



## Link to SyrNT

We can only hope to find connections based on the Syriac.
Let's see if there are words in the SyrNT text that show up in the persons and places lists.
First we try the lexemes.
Then the word occurrences.

In [56]:
hits = {dataType: {} for dataType in SC_TYPES}

for lx in F.otype.s('lexeme'):
    lex = F.lexeme.v(lx)
    for dataType in SC_TYPES:
        idV = tables[dataType][syriacF].get(lex, None)
        if idV is not None:
            hits[dataType][lx] = idV
    

In [57]:
for (dataType, theseHits) in hits.items():
    print(f'{dataType:>12}s: {len(theseHits):>5} hits')

      persons:    98 hits
       places:    37 hits


We show the hits by picking the first occurrence of each lexeme and showing it in context.

In [59]:
for (dataType, theseHits) in hits.items():
    markdown = f'''### {dataType}s
lexeme | linked | n-occs | passage | verse text
--- | --- | --- | --- | ---
'''
    for (lx, linked) in sorted(theseHits.items()):
        lex = F.lexeme.v(lx)
        ids = ' '.join(sorted(linked))
        occs = L.d(lx, otype='word')
        passage = '{} {}:{}'.format(*T.sectionFromNode(occs[0]))
        verse = L.u(occs[0], otype='verse')[0]
        text = T.text(L.d(verse, otype='word'))
        markdown += f'{lex} | {ids} | {len(occs)} | {passage} | {text}\n'
    display(Markdown(markdown))

### persons
lexeme | linked | n-occs | passage | verse text
--- | --- | --- | --- | ---
ܐܒܪܗܡ | 1108 1109 1110 1546 1547 1548 1549 1551 1552 1553 1554 2202 964 | 2 | Matthew 1:1 | ܟܬܒܐ ܕܝܠܝܕܘܬܗ ܕܝܫܘܥ ܡܫܝܚܐ ܒܪܗ ܕܕܘܝܕ ܒܪܗ ܕܐܒܪܗܡ 
ܐܝܣܚܩ | 1788 1789 1790 1791 1792 2578 | 3 | Matthew 1:2 | ܐܒܪܗܡ ܐܘܠܕ ܠܐܝܣܚܩ ܐܝܣܚܩ ܐܘܠܕ ܠܝܥܩܘܒ ܝܥܩܘܒ ܐܘܠܕ ܠܝܗܘܕܐ ܘܠܐܚܘܗܝ 
ܝܥܩܘܒ | 1299 1301 1302 1797 1798 1800 962 | 3 | Matthew 1:2 | ܐܒܪܗܡ ܐܘܠܕ ܠܐܝܣܚܩ ܐܝܣܚܩ ܐܘܠܕ ܠܝܥܩܘܒ ܝܥܩܘܒ ܐܘܠܕ ܠܝܗܘܕܐ ܘܠܐܚܘܗܝ 
ܝܗܘܕܐ | 3125 | 1 | Matthew 1:2 | ܐܒܪܗܡ ܐܘܠܕ ܠܐܝܣܚܩ ܐܝܣܚܩ ܐܘܠܕ ܠܝܥܩܘܒ ܝܥܩܘܒ ܐܘܠܕ ܠܝܗܘܕܐ ܘܠܐܚܘܗܝ 
ܐܚܐ | 1122 1123 1740 | 3 | Matthew 1:2 | ܐܒܪܗܡ ܐܘܠܕ ܠܐܝܣܚܩ ܐܝܣܚܩ ܐܘܠܕ ܠܝܥܩܘܒ ܝܥܩܘܒ ܐܘܠܕ ܠܝܗܘܕܐ ܘܠܐܚܘܗܝ 
ܡܠܟܐ | 1350 1862 | 3 | Matthew 1:6 | ܐܝܫܝ ܐܘܠܕ ܠܕܘܝܕ ܡܠܟܐ ܕܘܝܕ ܐܘܠܕ ܠܫܠܝܡܘܢ ܡܢ ܐܢܬܬܗ ܕܐܘܪܝܐ 
ܫܠܝܡܘܢ | 2060 | 2 | Matthew 1:6 | ܐܝܫܝ ܐܘܠܕ ܠܕܘܝܕ ܡܠܟܐ ܕܘܝܕ ܐܘܠܕ ܠܫܠܝܡܘܢ ܡܢ ܐܢܬܬܗ ܕܐܘܪܝܐ 
ܐܡܘܢ | 1579 1580 | 3 | Matthew 1:10 | ܚܙܩܝܐ ܐܘܠܕ ܠܡܢܫܐ ܡܢܫܐ ܐܘܠܕ ܠܐܡܘܢ ܐܡܘܢ ܐܘܠܕ ܠܝܘܫܝܐ 
ܝܘܣܦ | 1836 1837 1838 2162 2475 | 1 | Matthew 1:16 | ܝܥܩܘܒ ܐܘܠܕ ܠܝܘܣܦ ܓܒܪܗ ܕܡܪܝܡ ܕܡܢܗ ܐܬܝܠܕ ܝܫܘܥ ܕܡܬܩܪܐ ܡܫܝܚܐ 
ܡܪܝܡ | 1880 624 | 4 | Matthew 1:16 | ܝܥܩܘܒ ܐܘܠܕ ܠܝܘܣܦ ܓܒܪܗ ܕܡܪܝܡ ܕܡܢܗ ܐܬܝܠܕ ܝܫܘܥ ܕܡܬܩܪܐ ܡܫܝܚܐ 
ܝܠܕܐ | 1052 | 4 | Matthew 1:18 | ܝܠܕܗ ܕܝܢ ܕܝܫܘܥ ܡܫܝܚܐ ܗܟܢܐ ܗܘܐ ܟܕ ܡܟܝܪܐ ܗܘܬ ܡܪܝܡ ܐܡܗ ܠܝܘܣܦ ܥܕܠܐ ܢܫܬܘܬܦܘܢ ܐܫܬܟܚܬ ܒܛܢܐ ܡܢ ܪܘܚܐ ܕܩܘܕܫܐ 
ܥܡܢܘܐܝܠ | 473 | 1 | Matthew 1:23 | ܕܗܐ ܒܬܘܠܬܐ ܬܒܛܢ ܘܬܐܠܕ ܒܪܐ ܘܢܩܪܘܢ ܫܡܗ ܥܡܢܘܐܝܠ ܕܡܬܬܪܓܡ ܥܡܢ ܐܠܗܢ 
ܛܠܝܐ | 1494 | 1 | Matthew 2:8 | ܘܫܕܪ ܐܢܘܢ ܠܒܝܬ-ܠܚܡ ܘܐܡܪ ܠܗܘܢ ܙܠܘ ܥܩܒܘ ܥܠ ܛܠܝܐ ܚܦܝܛܐܝܬ ܘܡܐ ܕܐܫܟܚܬܘܢܝܗܝ ܬܘ ܚܘܐܘܢܝ ܕܐܦ ܐܢܐ ܐܙܠ ܐܣܓܘܕ ܠܗ 
ܚܘܐ | 1731 | 2 | Matthew 2:8 | ܘܫܕܪ ܐܢܘܢ ܠܒܝܬ-ܠܚܡ ܘܐܡܪ ܠܗܘܢ ܙܠܘ ܥܩܒܘ ܥܠ ܛܠܝܐ ܚܦܝܛܐܝܬ ܘܡܐ ܕܐܫܟܚܬܘܢܝܗܝ ܬܘ ܚܘܐܘܢܝ ܕܐܦ ܐܢܐ ܐܙܠ ܐܣܓܘܕ ܠܗ 
ܐܠܝܐ | 1698 1699 1700 1703 1704 1705 2541 3145 945 | 1 | Matthew 2:18 | ܩܠܐ ܐܫܬܡܥ ܒܪܡܬܐ ܒܟܝܐ ܘܐܠܝܐ ܣܓܝܐܐ ܪܚܝܠ ܒܟܝܐ ܥܠ ܒܢܝܗ ܘܠܐ ܨܒܝܐ ܠܡܬܒܝܐܘ ܡܛܠ ܕܠܐ ܐܝܬܝܗܘܢ 
ܐܒܐ | 1094 2582 308 | 9 | Matthew 2:22 | ܟܕ ܕܝܢ ܫܡܥ ܕܐܪܟܠܐܘܣ ܗܘܐ ܡܠܟܐ ܒܝܗܘܕ ܚܠܦ ܗܪܘܕܣ ܐܒܘܗܝ ܕܚܠ ܕܢܐܙܠ ܠܬܡܢ ܘܐܬܚܙܝ ܠܗ ܒܚܠܡܐ ܕܢܐܙܠ ܠܐܬܪܐ ܕܓܠܝܠܐ 
ܝܘܚܢܢ | 1321 1801 1802 1803 1805 1806 1811 1813 1818 1820 1821 1826 2400 2424 2449 2477 2683 2685 2686 2787 854 977 | 1 | Matthew 3:1 | ܒܗܘܢ ܕܝܢ ܒܝܘܡܬܐ ܗܢܘܢ ܐܬܐ ܝܘܚܢܢ ܡܥܡܕܢܐ ܘܡܟܪܙ ܗܘܐ ܒܚܘܪܒܐ ܕܝܗܘܕ 
ܐܫܥܝܐ | 1793 2496 3141 | 5 | Matthew 3:3 | ܗܢܘ ܓܝܪ ܗܘ ܕܐܡܝܪ ܒܝܕ ܐܫܥܝܐ ܢܒܝܐ ܩܠܐ ܕܩܪܐ ܒܚܘܪܒܐ ܛܝܒܘ ܐܘܪܚܗ ܕܡܪܝܐ ܘܐܫܘܘ ܠܫܒܝܠܘܗܝ 
ܫܡܥܘܢ | 1471 1474 1475 2093 2094 2095 2100 986 | 3 | Matthew 4:18 | ܘܟܕ ܡܗܠܟ ܥܠ ܝܕ ܝܡܐ ܕܓܠܝܠܐ ܚܙܐ ܬܪܝܢ ܐܚܝܢ ܫܡܥܘܢ ܕܐܬܩܪܝ ܟܐܦܐ ܘܐܢܕܪܐܘܣ ܐܚܘܗܝ ܕܪܡܝܢ ܡܨܝܕܬܐ ܒܝܡܐ ܐܝܬܝܗܘܢ ܗܘܘ ܓܝܪ ܨܝܕܐ 
ܐܢܕܪܐܘܣ | 2205 891 954 | 3 | Matthew 4:18 | ܘܟܕ ܡܗܠܟ ܥܠ ܝܕ ܝܡܐ ܕܓܠܝܠܐ ܚܙܐ ܬܪܝܢ ܐܚܝܢ ܫܡܥܘܢ ܕܐܬܩܪܝ ܟܐܦܐ ܘܐܢܕܪܐܘܣ ܐܚܘܗܝ ܕܪܡܝܢ ܡܨܝܕܬܐ ܒܝܡܐ ܐܝܬܝܗܘܢ ܗܘܘ ܓܝܪ ܨܝܕܐ 
ܡܣܟܢܐ | 1988 | 2 | Matthew 5:3 | ܛܘܒܝܗܘܢ ܠܡܣܟܢܐ ܒܪܘܚ ܕܕܝܠܗܘܢ ܗܝ ܡܠܟܘܬܐ ܕܫܡܝܐ 
ܝܪܬ | 1509 | 5 | Matthew 5:5 | ܛܘܒܝܗܘܢ ܠܡܟܝܟܐ ܕܗܢܘܢ ܢܐܪܬܘܢ ܠܐܪܥܐ 
ܥܒܕܐ | 1097 1099 | 2 | Matthew 5:16 | ܗܟܢܐ ܢܢܗܪ ܢܘܗܪܟܘܢ ܩܕܡ ܒܢܝܢܫܐ ܕܢܚܙܘܢ ܥܒܕܝܟܘܢ ܛܒܐ ܘܢܫܒܚܘܢ ܠܐܒܘܟܘܢ ܕܒܫܡܝܐ 
ܙܥܘܪܐ | 1524 | 4 | Matthew 5:19 | ܟܠ ܡܢ ܗܟܝܠ ܕܢܫܪܐ ܚܕ ܡܢ ܦܘܩܕܢܐ ܗܠܝܢ ܙܥܘܪܐ ܘܢܠܦ ܗܟܢܐ ܠܒܢܝܢܫܐ ܒܨܝܪܐ ܢܬܩܪܐ ܒܡܠܟܘܬܐ ܕܫܡܝܐ ܟܠ ܕܝܢ ܕܢܥܒܕ ܘܢܠܦ ܗܢܐ ܪܒܐ ܢܬܩܪܐ ܒܡܠܟܘܬܐ ܕܫܡܝܐ 
ܫܡܘܢܐ | 1486 2078 | 5 | Matthew 5:26 | ܘܐܡܝܢ ܐܡܪ ܐܢܐ ܠܟ ܕܠܐ ܬܦܘܩ ܡܢ ܬܡܢ ܥܕܡܐ ܕܬܬܠ ܫܡܘܢܐ ܐܚܪܝܐ 
ܢܘܢܐ | 2292 | 3 | Matthew 7:10 | ܘܐܢ ܢܘܢܐ ܢܫܐܠܝܘܗܝ ܠܡܐ ܚܘܝܐ ܡܘܫܛ ܠܗ 
ܚܠܐ | 1769 | 12 | Matthew 7:26 | ܘܟܠ ܡܢ ܕܫܡܥ ܡܠܝ ܗܠܝܢ ܘܠܐ ܥܒܕ ܠܗܝܢ ܢܬܕܡܐ ܠܓܒܪܐ ܣܟܠܐ ܕܒܢܐ ܒܝܬܗ ܥܠ ܚܠܐ 
ܡܬܝ | 1909 | 13 | Matthew 9:9 | ܘܟܕ ܥܒܪ ܝܫܘܥ ܡܢ ܬܡܢ ܚܙܐ ܓܒܪܐ ܕܝܬܒ ܒܝܬ ܡܟܣܐ ܕܫܡܗ ܡܬܝ ܘܐܡܪ ܠܗ ܬܐ ܒܬܪܝ ܘܩܡ ܐܙܠ ܒܬܪܗ 
ܐܣܝܐ | 1616 | 8 | Matthew 9:12 | ܝܫܘܥ ܕܝܢ ܟܕ ܫܡܥ ܐܡܪ ܠܗܘܢ ܠܐ ܣܢܝܩܝܢ ܚܠܝܡܐ ܥܠ ܐܣܝܐ ܐܠܐ ܐܝܠܝܢ ܕܒܝܫܐܝܬ ܥܒܝܕܝܢ 
ܡܐܢܐ | 2705 | 26 | Matthew 9:21 | ܐܡܪܐ ܗܘܬ ܓܝܪ ܒܢܦܫܗ ܐܦܢ ܒܠܚܘܕ ܠܡܐܢܗ ܩܪܒܐ ܐܢܐ ܡܬܐܣܝܐ ܐܢܐ 
ܦܝܠܝܦܘܣ | 1998 951 | 1 | Matthew 10:3 | ܘܦܝܠܝܦܘܣ ܘܒܪ-ܬܘܠܡܝ ܘܬܐܘܡܐ ܘܡܬܝ ܡܟܣܐ ܘܝܥܩܘܒ ܒܪ ܚܠܦܝ ܘܠܒܝ ܕܐܬܟܢܝ ܬܕܝ 
ܬܐܘܡܐ | 2123 2125 2126 2127 2586 2772 979 | 5 | Matthew 10:3 | ܘܦܝܠܝܦܘܣ ܘܒܪ-ܬܘܠܡܝ ܘܬܐܘܡܐ ܘܡܬܝ ܡܟܣܐ ܘܝܥܩܘܒ ܒܪ ܚܠܦܝ ܘܠܒܝ ܕܐܬܟܢܝ ܬܕܝ 
ܢܘܚ | 1949 3022 | 13 | Matthew 10:15 | ܘܐܡܝܢ ܐܡܪ ܐܢܐ ܠܟܘܢ ܕܠܐܪܥܐ ܕܣܕܘܡ ܘܕܥܡܘܪܐ ܢܗܘܐ ܢܝܚ ܒܝܘܡܐ ܕܕܝܢܐ ܐܘ ܠܡܕܝܢܬܐ ܗܝ 
ܝܘܢܢ | 1513 1830 | 6 | Matthew 12:39 | ܗܘ ܕܝܢ ܥܢܐ ܘܐܡܪ ܠܗܘܢ ܫܪܒܬܐ ܒܝܫܬܐ ܘܓܝܪܬܐ ܐܬܐ ܒܥܝܐ ܘܐܬܐ ܠܐ ܬܬܝܗܒ ܠܗ ܐܠܐ ܐܬܗ ܕܝܘܢܢ ܢܒܝܐ 
ܚܬܐ | 3030 | 33 | Matthew 12:50 | ܟܠܢܫ ܓܝܪ ܕܥܒܕ ܨܒܝܢܗ ܕܐܒܝ ܕܒܫܡܝܐ ܗܘܝܘ ܐܚܝ ܘܚܬܝ ܘܐܡܝ 
ܙܢܝܬܐ | 1658 | 39 | Matthew 21:31 | ܡܢܘ ܡܢ ܗܠܝܢ ܬܪܝܗܘܢ ܥܒܕ ܨܒܝܢܐ ܕܐܒܘܗܝ ܐܡܪܝܢ ܠܗ ܗܘ ܩܕܡܝܐ ܐܡܪ ܠܗܘܢ ܝܫܘܥ ܐܡܝܢ ܐܡܪ ܐܢܐ ܠܟܘܢ ܕܡܟܣܐ ܘܙܢܝܬܐ ܩܕܡܝܢ ܠܟܘܢ ܠܡܠܟܘܬܐ ܕܐܠܗܐ 
ܗܒܝܠ | 1540 1541 | 4 | Matthew 23:35 | ܐܝܟܢܐ ܕܢܐܬܐ ܥܠܝܟܘܢ ܟܠܗ ܕܡܐ ܕܙܕܝܩܐ ܕܐܬܐܫܕ ܥܠ ܐܪܥܐ ܡܢ ܕܡܗ ܕܗܒܝܠ ܙܕܝܩܐ ܘܥܕܡܐ ܠܕܡܗ ܕܙܟܪܝܐ ܒܪ ܒܪܟܝܐ ܗܘ ܕܩܛܠܬܘܢ ܒܝܢܝ ܗܝܟܠܐ ܠܡܕܒܚܐ 
ܙܟܪܝܐ | 2153 2155 2476 | 2 | Matthew 23:35 | ܐܝܟܢܐ ܕܢܐܬܐ ܥܠܝܟܘܢ ܟܠܗ ܕܡܐ ܕܙܕܝܩܐ ܕܐܬܐܫܕ ܥܠ ܐܪܥܐ ܡܢ ܕܡܗ ܕܗܒܝܠ ܙܕܝܩܐ ܘܥܕܡܐ ܠܕܡܗ ܕܙܟܪܝܐ ܒܪ ܒܪܟܝܐ ܗܘ ܕܩܛܠܬܘܢ ܒܝܢܝ ܗܝܟܠܐ ܠܡܕܒܚܐ 
ܕܢܝܐܝܠ | 1017 1200 1203 1671 1672 1673 | 28 | Matthew 24:15 | ܡܐ ܕܝܢ ܕܚܙܝܬܘܢ ܐܬܐ ܛܢܦܬܐ ܕܚܘܪܒܐ ܕܐܬܐܡܪ ܒܕܢܝܐܝܠ ܢܒܝܐ ܕܩܝܡܐ ܒܕܘܟܬܐ ܩܕܝܫܬܐ ܗܘ ܕܩܪܐ ܢܣܬܟܠ 
ܨܠܝܒܐ | 1661 | 10 | Mark 10:21 | ܝܫܘܥ ܕܝܢ ܚܪ ܒܗ ܘܐܚܒܗ ܘܐܡܪ ܠܗ ܚܕܐ ܚܣܝܪܐ ܠܟ ܙܠ ܙܒܢ ܟܠ ܡܕܡ ܕܐܝܬ ܠܟ ܘܗܒ ܠܡܣܟܢܐ ܘܬܗܘܐ ܠܟ ܣܝܡܬܐ ܒܫܡܝܐ ܘܣܒ ܨܠܝܒܐ ܘܬܐ ܒܬܪܝ 
ܐܠܟܣܢܕܪܘܣ | 1574 887 | 1 | Mark 15:21 | ܘܫܚܪܘ ܚܕ ܕܥܒܪ ܗܘܐ ܫܡܥܘܢ ܩܘܪܝܢܝܐ ܕܐܬܐ ܗܘܐ ܡܢ ܩܪܝܬܐ ܐܒܘܗܝ ܕܐܠܟܣܢܕܪܘܣ ܘܕܪܘܦܘܣ ܕܢܫܩܘܠ ܙܩܝܦܗ 
ܬܐܘܦܝܠܐ | 2812 | 17 | Luke 1:3 | ܐܬܚܙܝ ܐܦ ܠܝ ܡܛܠ ܕܩܪܝܒ ܗܘܝܬ ܝܨܝܦܐܝܬ ܠܟܠܗܘܢ ܕܟܠ ܡܕܡ ܒܛܟܣܗ ܐܟܬܘܒ ܠܟ ܢܨܝܚܐ ܬܐܘܦܝܠܐ 
ܐܗܪܘܢ | 1012 1092 1533 1534 | 3 | Luke 1:5 | ܗܘܐ ܒܝܘܡܬܗ ܕܗܪܘܕܣ ܡܠܟܐ ܕܝܗܘܕܐ ܟܗܢܐ ܚܕ ܕܫܡܗ ܗܘܐ ܙܟܪܝܐ ܡܢ ܬܫܡܫܬܐ ܕܒܝܬ ܐܒܝܐ ܘܐܢܬܬܗ ܡܢ ܒܢܬܗ ܕܐܗܪܘܢ ܫܡܗ ܗܘܐ ܐܠܝܫܒܥ 
ܣܒܐ | 2045 2046 2047 | 11 | Luke 1:18 | ܘܐܡܪ ܙܟܪܝܐ ܠܡܠܐܟܐ ܐܝܟܢܐ ܐܕܥ ܗܕܐ ܐܢܐ ܓܝܪ ܐܝܬܝ ܣܒܐ ܘܐܢܬܬܝ ܣܓܝܐܬ ܒܝܘܡܬܗ ܗܝ 
ܕܢܚܐ | 160 444 97 | 2 | Luke 1:78 | ܒܪܚܡܐ ܕܚܢܢܐ ܕܐܠܗܢ ܕܒܗܘܢ ܢܣܥܪܢ ܕܢܚܐ ܡܢ ܪܘܡܐ 
ܚܢܐ | 1590 | 4 | Luke 2:36 | ܘܚܢܐ ܕܝܢ ܢܒܝܬܐ ܒܪܬܗ ܕܦܢܘܐܝܠ ܡܢ ܫܒܛܐ ܕܐܫܝܪ ܐܦ ܗܝ ܩܫܝܫܬ ܒܝܘܡܬܗ ܗܘܬ ܘܫܒܥ ܫܢܝܢ ܥܡ ܒܥܠܗ ܚܝܬ ܡܢ ܒܬܘܠܘܬܗ 
ܐܕܝ | 1117 1118 2203 | 2 | Luke 3:28 | ܒܪ ܡܠܟܝ ܒܪ ܐܕܝ ܒܪ ܩܘܣܡ ܒܪ ܐܠܡܘܕܕ ܒܪ ܥܝܪ 
ܡܐܢܝ | 608 | 2 | Luke 3:31 | ܒܪ ܡܠܝܐ ܒܪ ܡܐܢܝ ܒܪ ܡܛܬܐ ܒܪ ܢܬܢ ܒܪ ܕܘܝܕ 
ܐܕܡ | 1560 | 208 | Luke 3:38 | ܒܪ ܐܢܘܫ ܒܪ ܫܝܬ ܒܪ ܐܕܡ ܕܡܢ ܐܠܗܐ 
ܝܚܝܕܝܐ | 2088 | 183 | Luke 7:12 | ܘܟܕ ܩܪܒ ܠܬܪܥܐ ܕܡܕܝܢܬܐ ܚܙܐ ܟܕ ܡܠܘܝܢ ܡܝܬܐ ܕܝܚܝܕܝܐ ܗܘܐ ܠܐܡܗ ܘܗܝ ܐܡܗ ܐܪܡܠܬܐ ܗܘܬ ܘܟܢܫܐ ܣܓܝܐܐ ܕܒܢܝ ܡܕܝܢܬܐ ܥܡܗ 
ܫܘܫܢ | 1168 2092 | 46 | Luke 8:3 | ܘܝܘܚܢ ܐܢܬܬ ܟܘܙܐ ܪܒܝܬܗ ܕܗܪܘܕܣ ܘܫܘܫܢ ܘܐܚܪܢܝܬܐ ܣܓܝܐܬܐ ܐܝܠܝܢ ܕܡܫܡܫܢ ܗܘܝ ܠܗܘܢ ܡܢ ܩܢܝܢܝܗܝܢ 
ܡܪܬܐ | 1370 | 137 | Luke 10:38 | ܘܗܘܐ ܕܟܕ ܗܢܘܢ ܪܕܝܢ ܒܐܘܪܚܐ ܥܠ ܠܩܪܝܬܐ ܚܕܐ ܘܐܢܬܬܐ ܕܫܡܗ ܡܪܬܐ ܩܒܠܬܗ ܒܒܝܬܗ 
ܠܥܙܪ | 1852 | 9 | Luke 16:20 | ܘܡܣܟܢܐ ܚܕ ܐܝܬ ܗܘܐ ܕܫܡܗ ܠܥܙܪ ܘܪܡܐ ܗܘܐ ܠܘܬ ܬܪܥܗ ܕܗܘ ܥܬܝܪܐ ܟܕ ܡܡܚܝ ܒܫܘܚܢܐ 
ܙܟܝ | 2156 | 26 | Luke 19:2 | ܓܒܪܐ ܚܕ ܕܫܡܗ ܙܟܝ ܥܬܝܪܐ ܗܘܐ ܘܪܒ ܡܟܣܐ 
ܥܡܐܘܣ | 91 | 56 | Luke 24:13 | ܘܗܐ ܬܪܝܢ ܡܢܗܘܢ ܒܗ ܒܝܘܡܐ ܐܙܠܝܢ ܗܘܘ ܠܩܪܝܬܐ ܕܫܡܗ ܥܡܐܘܣ ܘܦܪܝܩܐ ܡܢ ܐܘܪܫܠܡ ܐܣܛܕܘܬܐ ܫܬܝܢ 
ܢܬܢܝܐܝܠ | 1942 651 | 116 | John 1:45 | ܘܦܝܠܝܦܘܣ ܐܫܟܚ ܠܢܬܢܝܐܝܠ ܘܐܡܪ ܠܗ ܗܘ ܕܟܬܒ ܥܠܘܗܝ ܡܘܫܐ ܒܢܡܘܣܐ ܘܒܢܒܝܐ ܐܫܟܚܢܝܗܝ ܕܝܫܘܥ ܗܘ ܒܪ ܝܘܣܦ ܡܢ ܢܨܪܬ 
ܣܪܐ | 2063 | 65 | John 11:39 | ܘܐܡܪ ܝܫܘܥ ܫܩܘܠܘ ܟܐܦܐ ܗܕܐ ܐܡܪܐ ܠܗ ܡܪܬܐ ܚܬܗ ܕܗܘ ܡܝܬܐ ܡܪܝ ܡܢ ܟܕܘ ܣܪܝ ܠܗ ܐܪܒܥܐ ܠܗ ܓܝܪ ܝܘܡܝܢ 
ܐܦܪܝܡ | 1098 13 | 99 | John 11:54 | ܗܘ ܕܝܢ ܝܫܘܥ ܠܐ ܡܗܠܟ ܗܘܐ ܓܠܝܐܝܬ ܒܝܬ ܝܗܘܕܝܐ ܐܠܐ ܐܙܠ ܠܗ ܡܢ ܬܡܢ ܠܐܬܪܐ ܕܩܪܝܒ ܠܚܘܪܒܐ ܠܟܪܟܐ ܕܡܬܩܪܐ ܐܦܪܝܡ ܘܬܡܢ ܡܬܗܦܟ ܗܘܐ ܥܡ ܬܠܡܝܕܘܗܝ 
ܛܘܒܢܐ | 120 | 25 | John 13:17 | ܐܢ ܗܠܝܢ ܬܕܥܘܢ ܛܘܒܢܐ ܐܢܬܘܢ ܐܢ ܬܥܒܕܘܢ ܐܢܝܢ 
ܦܛܪܘܣ | 2005 2008 2011 2409 2733 | 19 | Acts 1:13 | ܘܡܢ ܒܬܪ ܕܥܠܘ ܣܠܩܘ ܠܗܘܢ ܠܥܠܝܬܐ ܗܝ ܕܗܘܝܢ ܗܘܘ ܒܗ ܦܛܪܘܣ ܘܝܘܚܢܢ ܘܝܥܩܘܒ ܘܐܢܕܪܐܘܣ ܘܦܝܠܝܦܘܣ ܘܬܐܘܡܐ ܘܡܬܝ ܘܒܪ-ܬܘܠܡܝ ܘܝܥܩܘܒ ܒܪ ܚܠܦܝ ܘܫܡܥܘܢ ܛܢܢܐ ܘܝܗܘܕܐ ܒܪ ܝܥܩܘܒ 
ܥܙܝܙܐ | 1531 | 97 | Acts 2:2 | ܗܘܐ ܡܢ ܫܠܝܐ ܡܢ ܫܡܝܐ ܩܠܐ ܐܝܟ ܪܘܚܐ ܥܙܝܙܬܐ ܘܐܬܡܠܝ ܗܘܐ ܡܢܗ ܟܠܗ ܒܝܬܐ ܗܘ ܕܒܗ ܝܬܒܝܢ ܗܘܘ 
ܒܪܢܒܐ | 2616 | 9 | Acts 4:36 | ܝܘܣܦ ܕܝܢ ܗܘ ܕܐܬܟܢܝ ܒܪܢܒܐ ܡܢ ܫܠܝܚܐ ܕܡܬܬܪܓܡ ܒܪܐ ܕܒܘܝܐܐ ܠܘܝܐ ܡܢ ܐܬܪܐ ܕܩܘܦܪܣ 
ܩܘܦܪܘܣ | 1657 | 16 | Acts 4:36 | ܝܘܣܦ ܕܝܢ ܗܘ ܕܐܬܟܢܝ ܒܪܢܒܐ ܡܢ ܫܠܝܚܐ ܕܡܬܬܪܓܡ ܒܪܐ ܕܒܘܝܐܐ ܠܘܝܐ ܡܢ ܐܬܪܐ ܕܩܘܦܪܣ 
ܚܢܢܝܐ | 1262 1581 1582 | 146 | Acts 5:1 | ܘܓܒܪܐ ܚܕ ܕܫܡܗ ܗܘܐ ܚܢܢܝܐ ܥܡ ܐܢܬܬܗ ܕܫܡܗ ܗܘܐ ܫܦܝܪܐ ܙܒܢ ܗܘܐ ܩܪܝܬܗ 
ܐܣܛܦܢܘܣ | 1229 1710 | 8 | Acts 6:5 | ܘܫܦܪܬ ܗܕܐ ܡܠܬܐ ܩܕܡ ܟܠܗ ܥܡܐ ܘܓܒܘ ܠܐܣܛܦܢܘܣ ܓܒܪܐ ܕܡܠܐ ܗܘܐ ܗܝܡܢܘܬܐ ܘܪܘܚܐ ܕܩܘܕܫܐ ܘܠܦܝܠܝܦܘܣ ܘܠܦܪܟܪܘܣ ܘܠܢܝܩܢܘܪ ܘܠܛܝܡܘܢ ܘܠܦܪܡܢܐ ܘܠܢܝܩܠܐܘܣ ܓܝܘܪܐ ܐܢܛܝܘܟܝܐ 
ܢܝܩܠܐܘܣ | 1946 | 2 | Acts 6:5 | ܘܫܦܪܬ ܗܕܐ ܡܠܬܐ ܩܕܡ ܟܠܗ ܥܡܐ ܘܓܒܘ ܠܐܣܛܦܢܘܣ ܓܒܪܐ ܕܡܠܐ ܗܘܐ ܗܝܡܢܘܬܐ ܘܪܘܚܐ ܕܩܘܕܫܐ ܘܠܦܝܠܝܦܘܣ ܘܠܦܪܟܪܘܣ ܘܠܢܝܩܢܘܪ ܘܠܛܝܡܘܢ ܘܠܦܪܡܢܐ ܘܠܢܝܩܠܐܘܣ ܓܝܘܪܐ ܐܢܛܝܘܟܝܐ 
ܣܝܢܝ | 3031 | 54 | Acts 7:30 | ܘܟܕ ܡܠܝ ܠܗ ܬܡܢ ܐܪܒܥܝܢ ܫܢܝܢ ܐܬܚܙܝ ܠܗ ܒܡܕܒܪܐ ܕܛܘܪ ܣܝܢܝ ܡܠܐܟܗ ܕܡܪܝܐ ܒܢܘܪܐ ܕܝܩܕܐ ܒܣܢܝܐ 
ܩܣܪܝܐ | 1641 | 14 | Acts 8:40 | ܦܝܠܝܦܘܣ ܕܝܢ ܐܫܬܟܚ ܒܐܙܘܛܘܣ ܘܡܢ ܬܡܢ ܡܬܟܪܟ ܗܘܐ ܘܡܣܒܪ ܒܡܕܝܢܬܐ ܟܠܗܝܢ ܥܕܡܐ ܕܐܬܐ ܠܩܣܪܝܐ 
ܡܪܩܘܣ | 1356 1869 1870 1872 2557 619 | 15 | Acts 12:12 | ܘܟܕ ܐܣܬܟܠ ܐܬܐ ܠܗ ܠܒܝܬܐ ܕܡܪܝܡ ܐܡܗ ܕܝܘܚܢܢ ܗܘ ܕܐܬܟܢܝ ܡܪܩܘܣ ܡܛܠ ܕܐܚܐ ܣܓܝܐܐ ܬܡܢ ܟܢܝܫܝܢ ܗܘܘ ܘܡܨܠܝܢ 
ܦܦܘܣ | 1965 | 13 | Acts 13:6 | ܘܟܕ ܐܬܟܪܟܘܗ ܠܟܠܗ ܓܙܪܬܐ ܥܕܡܐ ܠܦܦܘܣ ܡܕܝܢܬܐ ܐܫܟܚܘ ܓܒܪܐ ܚܕ ܚܪܫܐ ܝܗܘܕܝܐ ܕܐܝܬܘܗܝ ܗܘܐ ܢܒܝܐ ܕܓܠܐ ܕܫܡܗ ܗܘܐ ܒܪܫܘܡܐ 
ܦܘܠܘܣ | 1405 1976 1977 1978 1981 1983 1985 1986 1987 1990 3148 | 69 | Acts 13:9 | ܫܐܘܠ ܕܝܢ ܗܘ ܕܐܬܩܪܝ ܦܘܠܘܣ ܐܬܡܠܝ ܗܘܐ ܒܪܘܚܐ ܕܩܘܕܫܐ ܘܚܪ ܒܗ 
ܒܢܝܡܝܢ | 1172 1632 | 245 | Acts 13:21 | ܘܗܝܕܝܢ ܫܐܠܘ ܠܗܘܢ ܡܠܟܐ ܘܝܗܒ ܠܗܘܢ ܐܠܗܐ ܠܫܐܘܠ ܒܪ ܩܝܫ ܓܒܪܐ ܡܢ ܫܪܒܬܐ ܕܒܢܝܡܝܢ ܫܢܝܢ ܐܪܒܥܝܢ 
ܛܝܡܬܐܘܣ | 2129 2130 2773 | 65 | Acts 16:1 | ܘܡܛܝ ܗܘܐ ܠܕܪܒܐ ܡܕܝܢܬܐ ܘܠܠܘܣܛܪܐ ܐܝܬ ܗܘܐ ܕܝܢ ܬܡܢ ܬܠܡܝܕܐ ܚܕ ܕܫܡܗ ܛܝܡܬܐܘܣ ܒܪܗ ܕܝܗܘܕܝܬܐ ܚܕܐ ܡܗܝܡܢܬܐ ܘܐܒܘܗܝ ܐܪܡܝܐ ܗܘܐ 
ܐܬܢܘܣ | 146 | 22 | Acts 17:15 | ܘܗܢܘܢ ܕܐܬܠܘܝܘ ܠܗ ܠܦܘܠܘܣ ܐܬܘ ܥܡܗ ܥܕܡܐ ܠܐܬܢܘܣ ܡܕܝܢܬܐ ܘܟܕ ܢܦܩܝܢ ܡܢ ܨܐܕܘܗܝ ܩܒܠܘ ܡܢܗ ܐܓܪܬܐ ܠܘܬ ܫܝܠܐ ܘܛܝܡܬܐܘܣ ܕܒܥܓܠ ܢܐܙܠܘܢ ܠܘܬܗ 
ܐܦܠܘ | 1599 1601 | 20 | Acts 18:24 | ܘܓܒܪܐ ܚܕ ܕܫܡܗ ܗܘܐ ܐܦܠܘ ܝܗܘܕܝܐ ܕܐܝܬܘܗܝ ܗܘܐ ܛܘܗܡܗ ܡܢ ܐܠܟܣܢܕܪܝܐ ܘܪܕܐ ܗܘܐ ܒܡܠܬܐ ܘܡܕܩ ܗܘܐ ܒܟܬܒܐ ܐܬܐ ܠܐܦܣܘܣ 
ܣܩܘܢܕܘܣ | 727 | 2 | Acts 20:4 | ܘܢܦܩܘ ܥܡܗ ܥܕܡܐ ܠܐܣܝܐ ܣܘܦܛܪܘܣ ܕܡܢ ܒܪܘܐܐ ܡܕܝܢܬܐ ܘܐܪܣܛܪܟܘܣ ܘܣܩܘܢܕܘܣ ܕܡܢ ܬܣܠܘܢܝܩܐ ܘܓܐܝܘܣ ܕܡܢ ܕܪܒܐ ܡܕܝܢܬܐ ܘܛܝܡܬܐܘܣ ܕܡܢ ܠܘܣܛܪܐ ܘܡܢ ܐܣܝܐ ܛܘܟܝܩܘܣ ܘܛܪܘܦܝܡܘܣ 
ܡܝܠܝܛܘܣ | 1919 | 51 | Acts 20:15 | ܘܡܢ ܬܡܢ ܠܝܘܡܐ ܐܚܪܢܐ ܪܕܝܢ ܠܘܩܒܠ ܟܝܘܣ ܓܙܪܬܐ ܘܬܘܒ ܠܝܘܡܐ ܐܚܪܢܐ ܐܬܝܢ ܠܣܡܘܣ ܘܩܘܝܢ ܒܛܪܘܓܠܝܘܢ ܘܠܝܘܡܐ ܐܚܪܢܐ ܐܬܝܢ ܠܡܝܠܝܛܘܣ 
ܦܐܛܪܐ | 2013 | 29 | Acts 21:1 | ܘܦܪܫܢ ܡܢܗܘܢ ܘܪܕܝܢ ܬܪܝܨܐܝܬ ܠܩܘ ܓܙܪܬܐ ܘܠܝܘܡܐ ܐܚܪܢܐ ܐܬܝܢ ܠܪܘܕܘܣ ܘܡܢ ܬܡܢ ܠܦܐܛܪܐ 
ܒܪܢܝܩܐ | 2618 | 65 | Acts 25:13 | ܘܟܕ ܗܘܘ ܝܘܡܬܐ ܢܚܬ ܐܓܪܦܘܣ ܡܠܟܐ ܘܒܪܢܝܩܐ ܠܩܣܪܝܐ ܕܢܫܐܠܘܢ ܫܠܡܗ ܕܦܗܣܛܘܣ 
ܝܘܠܝܘܣ | 2163 | 9 | Acts 27:1 | ܘܦܩܕ ܥܠܘܗܝ ܦܗܣܛܣ ܕܢܫܬܕܪ ܠܘܬ ܩܣܪ ܠܐܝܛܠܝܐ ܘܐܫܠܡܗ ܠܦܘܠܘܣ ܘܠܐܣܝܪܐ ܐܚܪܢܐ ܥܡܗ ܠܓܒܪܐ ܚܕ ܩܢܛܪܘܢܐ ܡܢ ܐܣܦܝܪ ܣܒܣܛܐ ܕܫܡܗ ܗܘܐ ܝܘܠܝܘܣ 
ܦܘܦܠܝܘܣ | 888 | 15 | Acts 28:7 | ܐܝܬ ܗܘܐ ܕܝܢ ܩܘܪܝܐ ܒܗ ܒܗܘ ܐܬܪܐ ܠܓܒܪܐ ܚܕ ܕܫܡܗ ܗܘܐ ܦܘܦܠܝܘܣ ܕܗܘ ܗܘܐ ܪܫܗ ܕܓܙܪܬܐ ܘܩܒܠܢ ܒܒܝܬܗ ܬܠܬܐ ܝܘܡܝܢ ܚܕܝܐܝܬ 
ܗܘܫܥ | 1952 | 39 | Romans 9:25 | ܐܝܟܢܐ ܕܐܦ ܒܗܘܫܥ ܐܡܪ ܕܐܩܪܐ ܠܐܝܠܝܢ ܕܠܐ ܗܘܘ ܥܡܝ ܥܡܐ ܕܝܠܝ ܘܠܠܐ ܐܬܪܚܡܬ ܐܬܪܚܡܬ 
ܐܢܕܪܘܢܝܩܘܣ | 1588 1589 | 14 | Romans 16:7 | ܫܐܠܘ ܒܫܠܡܐ ܕܐܢܕܪܘܢܝܩܘܣ ܘܕܝܘܢܝܐ ܐܚܝܢܝ ܕܗܘܘ ܫܒܝܐ ܥܡܝ ܘܝܕܝܥܝܢ ܐܢܘܢ ܒܫܠܝܚܐ ܘܒܡܫܝܚܐ ܩܕܡܝ ܗܘܘ 
ܐܣܛܦܢܐ | 1709 3010 | 55 | 1_Corinthians 1:16 | ܐܥܡܕܬ ܕܝܢ ܐܦ ܠܒܝܬܗ ܕܐܣܛܦܢܐ ܬܘܒ ܕܝܢ ܠܐ ܝܕܥ ܐܢܐ ܐܢ ܠܐܢܫ ܐܚܪܝܢ ܐܥܡܕܬ 
ܟܪܘܒܐ | 1644 | 1 | 1_Corinthians 9:10 | ܐܠܐ ܝܕܝܥܐ ܕܡܛܠܬܢ ܗܘ ܐܡܪ ܘܡܛܠܬܢ ܓܝܪ ܐܬܟܬܒܬ ܡܛܠ ܕܥܠ ܣܒܪܐ ܗܘ ܘܠܐ ܠܗ ܠܟܪܘܒܐ ܕܢܟܪܘܒ ܘܐܝܢܐ ܕܡܕܪܟ ܥܠ ܣܒܪܐ ܕܥܠܠܬܐ 
ܪܚܝܡܐ | 2036 | 8 | Philippians 4:1 | ܡܟܝܠ ܐܚܝ ܚܒܝܒܐ ܘܪܚܝܡܐ ܘܚܕܘܬܝ ܘܟܠܝܠܝ ܗܟܢܐ ܩܘܡܘ ܒܡܪܢ ܚܒܝܒܝ 
ܩܠܡܝܣ | 1648 423 | 52 | Philippians 4:3 | ܐܦ ܡܢܟ ܒܥܐ ܐܢܐ ܒܪ ܙܘܓܝ ܫܪܝܪܐ ܕܬܗܘܐ ܡܥܕܪ ܠܗܝܢ ܕܗܢܝܢ ܠܐܝ ܥܡܝ ܒܐܘܢܓܠܝܘܢ ܥܡ ܩܠܡܝܣ ܘܥܡ ܫܪܟܐ ܕܡܥܕܪܢܝ ܗܢܘܢ ܕܫܡܗܝܗܘܢ ܟܬܝܒܝܢ ܒܟܬܒܐ ܕܚܝܐ 
ܐܢܣܝܡܘܣ | 1950 | 18 | Colossians 4:9 | ܥܡ ܐܢܣܝܡܘܣ ܐܚܐ ܡܗܝܡܢܐ ܘܚܒܝܒܐ ܗܘ ܕܐܝܬܘܗܝ ܡܢܟܘܢ ܗܢܘܢ ܢܘܕܥܘܢܟܘܢ ܡܕܡ ܕܠܘܬܢ 
ܕܕܐ | 1667 | 56 | Colossians 4:10 | ܫܐܠ ܒܫܠܡܟܘܢ ܐܪܣܛܪܟܘܣ ܫܒܝܐ ܕܥܡܝ ܘܡܪܩܘܣ ܒܪ ܕܕܗ ܕܒܪܢܒܐ ܗܘ ܕܐܬܦܩܕܬܘܢ ܥܠܘܗܝ ܕܐܢ ܐܬܐ ܠܘܬܟܘܢ ܬܩܒܠܘܢܝܗܝ 
ܠܘܩܐ | 1855 | 3 | Colossians 4:14 | ܫܐܠ ܒܫܠܡܟܘܢ ܠܘܩܐ ܐܣܝܐ ܚܒܝܒܢ ܘܕܡܐ 
ܐܘܒܘܠܘܣ | 3028 | 1 | 2_Timothy 4:21 | ܢܬܒܛܠ ܠܟ ܕܩܕܡ ܣܬܘܐ ܬܐܬܐ ܫܐܠ ܒܫܠܡܟ ܐܘܒܘܠܘܣ ܘܦܘܕܣ ܘܠܝܢܘܣ ܘܩܠܘܕܝܐ ܘܐܚܐ ܟܠܗܘܢ 
ܦܝܠܝܡܘܢ | 1996 | 4 | Philemon 1:1 | ܦܘܠܘܣ ܐܣܝܪܗ ܕܝܫܘܥ ܡܫܝܚܐ ܘܛܝܡܬܐܘܣ ܐܚܐ ܠܦܝܠܡܘܢ ܚܒܝܒܐ ܘܦܠܚܐ ܕܥܡܢ 
ܩܐܝܢ | 1635 | 15 | Hebrews 11:4 | ܒܗܝܡܢܘܬܐ ܩܪܒ ܗܒܝܠ ܕܒܚܬܐ ܕܡܝܬܪܐ ܛܒ ܡܢ ܕܩܐܝܢ ܠܐܠܗܐ ܘܡܛܠܬܗ ܗܘܬ ܥܠܘܗܝ ܣܗܕܘܬܐ ܕܟܐܢܐ ܗܘ ܘܣܗܕ ܥܠ ܩܘܪܒܢܗ ܐܠܗܐ ܘܡܛܠܬܗ ܐܦ ܟܕ ܡܝܝܬ ܡܡܠܠ 
ܫܡܫܘܢ | 2061 2062 | 2 | Hebrews 11:32 | ܘܡܢܐ ܬܘܒ ܐܡܪ ܙܥܘܪ ܗܘ ܠܝ ܓܝܪ ܙܒܢܐ ܕܐܫܬܥܐ ܥܠ ܓܕܥܘܢ ܘܥܠ ܒܪܩ ܘܥܠ ܫܡܫܘܢ ܘܥܠ ܢܦܬܚ ܘܥܠ ܕܘܝܕ ܘܥܠ ܫܡܘܐܝܠ ܘܥܠ ܫܪܟܐ ܕܢܒܝܐ 
ܚܠܝܐ | 1031 | 16 | James 3:11 | ܕܠܡܐ ܡܫܟܚܐ ܕܡܢ ܚܕ ܡܒܘܥܐ ܢܦܩܘܢ ܡܝܐ ܚܠܝܐ ܘܡܪܝܪܐ 
ܡܝܟܐܝܠ | 1921 637 | 33 | Jude 1:9 | ܡܝܟܐܝܠ ܕܝܢ ܪܝܫ ܡܠܐܟܐ ܗܘ ܕܥܡ ܐܟܠܩܪܨܐ ܟܕ ܕܐܢ ܡܡܠܠ ܗܘܐ ܡܛܠ ܦܓܪܗ ܕܡܘܫܐ ܠܐ ܐܡܪܚ ܕܢܝܬܐ ܥܠܘܗܝ ܕܝܢܐ ܕܓܘܕܦܐ ܐܠܐ ܐܡܪ ܕܢܓܥܘܪ ܒܟ ܡܪܝܐ 
ܪܘܒܝܠ | 1432 | 5 | Revelation 7:5 | ܡܢ ܫܪܒܬܗ ܕܝܗܘܕܐ ܬܪܥܣܪ ܐܠܦܝܢ ܡܢ ܫܪܒܬܗ ܕܪܘܒܝܠ ܬܪܥܣܪ ܐܠܦܝܢ ܡܢ ܫܪܒܬܗ ܕܓܕ ܬܪܥܣܪ ܐܠܦܝܢ 
ܒܬܘܠܐ | 2145 | 37 | Revelation 14:4 | ܗܠܝܢ ܐܢܘܢ ܐܝܠܝܢ ܕܥܡ ܢܫܐ ܠܐ ܐܬܛܘܫܘ ܒܬܘܠܐ ܓܝܪ ܐܝܬܝܗܘܢ ܗܠܝܢ ܕܢܩܦܘܗܝ ܠܐܡܪܐ ܟܠ ܟܪ ܕܢܐܙܠ ܗܠܝܢ ܐܙܕܒܢܘ ܡܢ ܐܢܫܐ ܪܫܝܬܐ ܠܐܠܗܐ ܘܠܐܡܪܐ 


### places
lexeme | linked | n-occs | passage | verse text
--- | --- | --- | --- | ---
ܐܘܪܫܠܡ | 104 | 2 | Matthew 2:1 | ܟܕ ܕܝܢ ܐܬܝܠܕ ܝܫܘܥ ܒܒܝܬ-ܠܚܡ ܕܝܗܘܕܐ ܒܝܘܡܝ ܗܪܘܕܣ ܡܠܟܐ ܐܬܘ ܡܓܘܫܐ ܡܢ ܡܕܢܚܐ ܠܐܘܪܫܠܡ 
ܡܨܪܝܢ | 7 715 | 1 | Matthew 2:13 | ܟܕ ܕܝܢ ܐܙܠܘ ܐܬܚܙܝ ܡܠܐܟܐ ܕܡܪܝܐ ܒܚܠܡܐ ܠܝܘܣܦ ܘܐܡܪ ܠܗ ܩܘܡ ܕܒܪ ܠܛܠܝܐ ܘܠܐܡܗ ܘܥܪܘܩ ܠܡܨܪܝܢ ܘܬܡܢ ܗܘܝ ܥܕܡܐ ܕܐܡܪ ܐܢܐ ܠܟ ܥܬܝܕ ܗܘ ܓܝܪ ܗܪܘܕܣ ܠܡܒܥܝܗ ܠܛܠܝܐ ܐܝܟ ܕܢܘܒܕܝܘܗܝ 
ܚܡܬ | 91 | 3 | Matthew 2:16 | ܗܝܕܝܢ ܗܪܘܕܣ ܟܕ ܚܙܐ ܕܐܬܒܙܚ ܡܢ ܡܓܘܫܐ ܐܬܚܡܬ ܛܒ ܘܫܕܪ ܩܛܠ ܛܠܝܐ ܟܠܗܘܢ ܕܒܝܬ-ܠܚܡ ܘܕܟܠܗܘܢ ܬܚܘܡܝܗ ܡܢ ܒܪ ܬܪܬܝܢ ܫܢܝܢ ܘܠܬܚܬ ܐܝܟ ܙܒܢܐ ܕܥܩܒ ܡܢ ܡܓܘܫܐ 
ܥܒܪܐ | 1490 | 5 | Matthew 4:15 | ܐܪܥܐ ܕܙܒܘܠܘܢ ܐܪܥܐ ܕܢܦܬܠܝ ܐܘܪܚܐ ܕܝܡܐ ܥܒܪܘܗܝ ܕܝܘܪܕܢܢ ܓܠܝܠܐ ܕܥܡܡܐ 
ܣܘܪܝܐ | 486 | 5 | Matthew 4:24 | ܘܐܫܬܡܥ ܛܒܗ ܒܟܠܗ ܣܘܪܝܐ ܘܩܪܒܘ ܠܗ ܟܠܗܘܢ ܐܝܠܝܢ ܕܒܝܫ ܒܝܫ ܥܒܝܕܝܢ ܒܟܘܪܗܢܐ ܡܫܚܠܦܐ ܘܐܝܠܝܢ ܕܐܠܝܨܝܢ ܒܬܫܢܝܩܐ ܘܕܝܘܢܐ ܘܕܒܪ ܐܓܪܐ ܘܡܫܪܝܐ ܘܐܣܝ ܐܢܘܢ 
ܣܟܐ | 1467 | 3 | Matthew 5:34 | ܐܢܐ ܕܝܢ ܐܡܪ ܐܢܐ ܠܟܘܢ ܠܐ ܬܐܡܘܢ ܣܟ ܠܐ ܒܫܡܝܐ ܕܟܘܪܣܝܐ ܗܘ ܕܐܠܗܐ 
ܚܘܪܐ | 1456 | 4 | Matthew 5:36 | ܐܦܠܐ ܒܪܫܟ ܬܐܡܐ ܕܠܐ ܡܫܟܚ ܐܢܬ ܠܡܥܒܕ ܒܗ ܡܢܬܐ ܚܕܐ ܕܣܥܪܐ ܐܘܟܡܬܐ ܐܘ ܚܘܪܬܐ 
ܥܩܪ | 25 | 7 | Matthew 13:29 | ܗܘ ܕܝܢ ܐܡܪ ܠܗܘܢ ܕܠܡܐ ܟܕ ܡܓܒܝܢ ܐܢܬܘܢ ܙܝܙܢܐ ܬܥܩܪܘܢ ܥܡܗܘܢ ܐܦ ܚܛܐ 
ܬܠܐ | 200 | 1 | Matthew 18:6 | ܘܟܠ ܕܢܟܫܠ ܠܚܕ ܡܢ ܗܠܝܢ ܙܥܘܪܐ ܕܡܗܝܡܢܝܢ ܒܝ ܦܩܚ ܗܘܐ ܠܗ ܕܬܗܘܐ ܬܠܝܐ ܪܚܝܐ ܕܚܡܪܐ ܒܨܘܪܗ ܘܡܛܒܥ ܒܥܘܡܩܘܗܝ ܕܝܡܐ 
ܨܘܪܐ | 4 513 | 8 | Matthew 18:6 | ܘܟܠ ܕܢܟܫܠ ܠܚܕ ܡܢ ܗܠܝܢ ܙܥܘܪܐ ܕܡܗܝܡܢܝܢ ܒܝ ܦܩܚ ܗܘܐ ܠܗ ܕܬܗܘܐ ܬܠܝܐ ܪܚܝܐ ܕܚܡܪܐ ܒܨܘܪܗ ܘܡܛܒܥ ܒܥܘܡܩܘܗܝ ܕܝܡܐ 
ܚܦܪ | 331 | 2 | Matthew 21:33 | ܫܡܥܘ ܐܚܪܢܐ ܡܬܠܐ ܓܒܪܐ ܚܕ ܐܝܬ ܗܘܐ ܡܪܐ ܒܝܬܐ ܘܢܨܒ ܟܪܡܐ ܘܐܚܕܪܗ ܣܝܓܐ ܘܚܦܪ ܒܗ ܡܥܨܪܬܐ ܘܒܢܐ ܒܗ ܡܓܕܠܐ ܘܐܘܚܕܗ ܠܦܠܚܐ ܘܚܙܩ 
ܕܪܐ | 67 | 32 | Matthew 21:44 | ܘܡܢ ܕܢܦܠ ܥܠ ܟܐܦܐ ܗܕܐ ܢܬܪܥܥ ܘܟܠ ܡܢ ܕܗܝ ܬܦܠ ܥܠܘܗܝ ܬܕܪܝܘܗܝ 
ܐܪܟ | 515 | 1 | Matthew 23:5 | ܘܟܠܗܘܢ ܥܒܕܝܗܘܢ ܥܒܕܝܢ ܕܢܬܚܙܘܢ ܠܒܢܝ ܐܢܫܐ ܡܦܬܝܢ ܓܝܪ ܬܦܠܝܗܘܢ ܘܡܘܪܟܝܢ ܬܟܠܬܐ ܕܡܪܛܘܛܝܗܘܢ 
ܣܪܘܓ | 48 | 2 | Luke 3:35 | ܒܪ ܣܪܘܓ ܒܪ ܐܪܥܘ ܒܪ ܦܠܓ ܒܪ ܥܒܪ ܒܪ ܫܠܚ 
ܦܪܬ | 82 | 5 | Acts 1:18 | ܗܢܘ ܕܩܢܐ ܠܗ ܩܪܝܬܐ ܡܢ ܐܓܪܐ ܕܚܛܝܬܐ ܘܢܦܠ ܥܠ ܐܦܘܗܝ ܥܠ ܐܪܥܐ ܘܐܬܦܪܬ ܡܢ ܡܨܥܬܗ ܘܐܬܐܫܕ ܟܠܗ ܓܘܝܗ 
ܦܪܘܓܝܐ | 1525 | 1 | Acts 2:10 | ܘܕܡܢ ܐܬܪܐ ܕܦܪܘܓܝܐ ܘܕܦܡܦܘܠܝܐ ܘܕܡܨܪܝܢ ܘܕܐܬܪܘܬܐ ܕܠܘܒܐ ܕܩܪܝܒܝܢ ܠܩܘܪܝܢܐ ܘܐܝܠܝܢ ܕܐܬܘ ܡܢ ܪܗܘܡܐ ܝܗܘܕܝܐ ܘܓܝܘܪܐ 
ܩܘܦܪܘܣ | 714 | 16 | Acts 4:36 | ܝܘܣܦ ܕܝܢ ܗܘ ܕܐܬܟܢܝ ܒܪܢܒܐ ܡܢ ܫܠܝܚܐ ܕܡܬܬܪܓܡ ܒܪܐ ܕܒܘܝܐܐ ܠܘܝܐ ܡܢ ܐܬܪܐ ܕܩܘܦܪܣ 
ܐܢܛܝܘܟܝܐ | 10 995 | 44 | Acts 6:5 | ܘܫܦܪܬ ܗܕܐ ܡܠܬܐ ܩܕܡ ܟܠܗ ܥܡܐ ܘܓܒܘ ܠܐܣܛܦܢܘܣ ܓܒܪܐ ܕܡܠܐ ܗܘܐ ܗܝܡܢܘܬܐ ܘܪܘܚܐ ܕܩܘܕܫܐ ܘܠܦܝܠܝܦܘܣ ܘܠܦܪܟܪܘܣ ܘܠܢܝܩܢܘܪ ܘܠܛܝܡܘܢ ܘܠܦܪܡܢܐ ܘܠܢܝܩܠܐܘܣ ܓܝܘܪܐ ܐܢܛܝܘܟܝܐ 
ܐܠܟܣܢܕܪܝܐ | 572 | 2 | Acts 6:9 | ܘܩܡܘ ܗܘܘ ܐܢܫܐ ܡܢ ܟܢܘܫܬܐ ܕܡܬܩܪܝܐ ܕܠܝܒܪܛܝܢܘ ܘܩܘܪܝܢܝܐ ܘܐܠܟܣܢܕܪܝܐ ܘܕܡܢ ܩܝܠܝܩܝܐ ܘܡܢ ܐܣܝܐ ܘܕܪܫܝܢ ܗܘܘ ܥܡ ܐܣܛܦܢܘܣ 
ܩܝܠܝܩܝܐ | 55 | 115 | Acts 6:9 | ܘܩܡܘ ܗܘܘ ܐܢܫܐ ܡܢ ܟܢܘܫܬܐ ܕܡܬܩܪܝܐ ܕܠܝܒܪܛܝܢܘ ܘܩܘܪܝܢܝܐ ܘܐܠܟܣܢܕܪܝܐ ܘܕܡܢ ܩܝܠܝܩܝܐ ܘܡܢ ܐܣܝܐ ܘܕܪܫܝܢ ܗܘܘ ܥܡ ܐܣܛܦܢܘܣ 
ܚܪܢ | 216 | 20 | Acts 7:2 | ܗܘ ܕܝܢ ܐܡܪ ܓܒܪܐ ܐܚܝܢ ܘܐܒܗܬܢ ܫܡܥܘ ܐܠܗܐ ܕܬܫܒܘܚܬܐ ܐܬܚܙܝ ܠܐܒܘܢ ܐܒܪܗܡ ܟܕ ܐܝܬܘܗܝ ܗܘܐ ܒܝܬ-ܢܗܪܝܢ ܥܕ ܠܐ ܢܐܬܐ ܢܥܡܪ ܒܚܪܢ 
ܩܣܪܝܐ | 60 | 14 | Acts 8:40 | ܦܝܠܝܦܘܣ ܕܝܢ ܐܫܬܟܚ ܒܐܙܘܛܘܣ ܘܡܢ ܬܡܢ ܡܬܟܪܟ ܗܘܐ ܘܡܣܒܪ ܒܡܕܝܢܬܐ ܟܠܗܝܢ ܥܕܡܐ ܕܐܬܐ ܠܩܣܪܝܐ 
ܕܪܡܣܘܩ | 66 | 24 | Acts 9:2 | ܘܫܐܠ ܠܗ ܐܓܪܬܐ ܡܢ ܪܒ ܟܗܢܐ ܕܢܬܠ ܠܗ ܠܕܪܡܣܘܩ ܠܟܢܘܫܬܐ ܕܐܢ ܗܘ ܕܢܫܟܚ ܕܪܕܝܢ ܒܗܕܐ ܐܘܪܚܐ ܓܒܪܐ ܐܘ ܢܫܐ ܢܐܣܘܪ ܢܝܬܐ ܐܢܘܢ ܠܐܘܪܫܠܡ 
ܛܪܣܘܣ | 196 | 107 | Acts 9:11 | ܘܡܪܢ ܐܡܪ ܠܗ ܩܘܡ ܙܠ ܠܫܘܩܐ ܕܡܬܩܪܐ ܬܪܝܨܐ ܘܒܥܝ ܒܒܝܬܐ ܕܝܗܘܕܐ ܠܫܐܘܠ ܕܐܝܬܘܗܝ ܡܢ ܛܪܣܘܣ ܡܕܝܢܬܐ ܗܐ ܓܝܪ ܟܕ ܗܘ ܡܨܠܐ 
ܡܩܕܘܢܝܐ | 512 | 29 | Acts 16:9 | ܘܒܚܙܘܐ ܕܠܠܝܐ ܐܬܚܙܝ ܠܦܘܠܘܣ ܐܝܟ ܓܒܪܐ ܚܕ ܡܩܕܘܢܝܐ ܕܩܐܡ ܘܒܥܐ ܡܢܗ ܟܕ ܐܡܪ ܕܬܐ ܠܡܩܕܘܢܝܐ ܘܥܕܪܝܢܝ 
ܠܘܕܝܐ | 1526 | 2 | Acts 16:14 | ܘܐܢܬܬܐ ܚܕܐ ܡܙܒܢܬ ܐܪܓܘܢܐ ܕܕܚܠܐ ܗܘܬ ܡܢ ܐܠܗܐ ܫܡܗ ܗܘܐ ܠܘܕܝܐ ܡܢ ܬܐܘܛܝܪܐ ܡܕܝܢܬܐ ܕܗܕܐ ܦܬܚ ܠܒܗ ܡܪܢ ܘܫܡܥܐ ܗܘܬ ܡܕܡ ܕܐܡܪ ܦܘܠܘܣ 
ܐܦܣܘܣ | 623 | 69 | Acts 18:19 | ܘܡܛܝܘ ܠܐܦܣܘܣ ܘܥܠ ܦܘܠܘܣ ܠܟܢܘܫܬܐ ܘܡܡܠܠ ܗܘܐ ܥܡ ܝܗܘܕܝܐ 
ܓܐܝܘܣ | 1494 | 1 | Acts 19:29 | ܘܐܫܬܓܫܬ ܟܠܗ ܡܕܝܢܬܐ ܘܪܗܛܘ ܐܟܚܕܐ ܘܐܙܠܘ ܠܬܐܛܪܘܢ ܘܚܛܦܘ ܐܘܒܠܘ ܥܡܗܘܢ ܠܓܐܝܘܣ ܘܠܐܪܣܛܪܟܘܣ ܓܒܪܐ ܡܩܕܘܢܝܐ ܒܢܝ ܠܘܝܬܗ ܕܦܘܠܘܣ 
ܪܘܕܘܣ | 1534 | 3 | Acts 21:1 | ܘܦܪܫܢ ܡܢܗܘܢ ܘܪܕܝܢ ܬܪܝܨܐܝܬ ܠܩܘ ܓܙܪܬܐ ܘܠܝܘܡܐ ܐܚܪܢܐ ܐܬܝܢ ܠܪܘܕܘܣ ܘܡܢ ܬܡܢ ܠܦܐܛܪܐ 
ܦܐܛܪܐ | 157 | 29 | Acts 21:1 | ܘܦܪܫܢ ܡܢܗܘܢ ܘܪܕܝܢ ܬܪܝܨܐܝܬ ܠܩܘ ܓܙܪܬܐ ܘܠܝܘܡܐ ܐܚܪܢܐ ܐܬܝܢ ܠܪܘܕܘܣ ܘܡܢ ܬܡܢ ܠܦܐܛܪܐ 
ܠܘܩܝܐ | 1533 | 21 | Acts 27:5 | ܘܥܒܪܢ ܝܡܐ ܕܩܝܠܝܩܝܐ ܘܕܦܡܦܘܠܝܐ ܘܡܛܝܢ ܠܡܘܪܐ ܡܕܝܢܬܐ ܕܠܘܩܝܐ 
ܐܣܦܣ | 288 | 5 | Romans 3:13 | ܩܒܪܐ ܦܬܝܚܐ ܓܓܪܬܗܘܢ ܘܠܫܢܝܗܘܢ ܢܟܘܠܬܢܝܢ ܘܚܡܬܐ ܕܐܣܦܣ ܬܚܝܬ ܣܦܘܬܗܘܢ 
ܠܕܝܩܝܐ | 118 | 53 | Colossians 2:1 | ܨܒܐ ܐܢܐ ܕܝܢ ܕܬܕܥܘܢ ܐܝܢܐ ܐܓܘܢܐ ܐܝܬ ܠܝ ܚܠܦܝܟܘܢ ܘܚܠܦ ܗܢܘܢ ܕܒܠܕܝܩܝܐ ܘܚܠܦ ܫܪܟܐ ܐܝܠܝܢ ܕܦܪܨܘܦܝ ܠܐ ܚܙܘ ܒܒܣܪ 
ܩܠܘܕܝܐ | 63 | 110 | 2_Timothy 4:21 | ܢܬܒܛܠ ܠܟ ܕܩܕܡ ܣܬܘܐ ܬܐܬܐ ܫܐܠ ܒܫܠܡܟ ܐܘܒܘܠܘܣ ܘܦܘܕܣ ܘܠܝܢܘܣ ܘܩܠܘܕܝܐ ܘܐܚܐ ܟܠܗܘܢ 
ܥܙܐ | 87 | 16 | Hebrews 11:37 | ܐܚܪܢܐ ܐܬܪܓܡܘ ܐܚܪܢܐ ܐܬܢܣܪܘ ܐܚܪܢܐ ܒܦܘܡܐ ܕܣܝܦܐ ܡܝܬܘ ܐܚܪܢܐ ܐܬܟܪܟܘ ܟܕ ܠܒܝܫܝܢ ܡܫܟܐ ܕܐܡܪܐ ܘܕܥܙܐ ܘܣܢܝܩܝܢ ܘܐܠܝܨܝܢ ܘܡܛܪܦܝܢ 
ܥܒܐ | 517 | 16 | James 3:5 | ܗܟܢܐ ܐܦ ܠܫܢܐ ܗܕܡܐ ܗܘ ܙܥܘܪܐ ܘܡܫܬܥܠܐ ܐܦ ܢܘܪܐ ܙܥܘܪܬܐ ܥܒܐ ܣܓܝܐܐ ܡܘܩܕܐ 
ܨܠܚ | 2 | 92 | 3_John 1:2 | ܚܒܝܒܢ ܒܟܠܡܕܡ ܡܨܠܐ ܐܢܐ ܥܠܝܟ ܕܬܨܠܚ ܘܬܗܘܐ ܚܠܝܡ ܐܝܟ ܡܐ ܕܡܨܠܚܐ ܢܦܫܟ 


# Text API

So far, we have mainly seen nodes and their numbers, and the names of node types.
You would almost forget that we are dealing with text.
So let's try to see some text.

In the same way as `F` gives access to feature data,
`T` gives access to the text.
That is also feature data, but you can tell Text-Fabric which features are specifically
carrying the text, and in return Text-Fabric offers you
a Text API: `T`.

## Formats
Syriac text can be represented in a number of ways:

* in transliteration, or in Syriac characters,
* showing the actual text or only the lexemes,

If you wonder where the information about text formats is stored: 
not in the program text-fabric, but in the data set.
It has a feature `otext`, which specifies the formats and which features
must be used to produce them. `otext` is the third special feature in a TF data set,
next to `otype` and `oslots`. 
It is an optional feature. 
If it is absent, there will be no `T` API.

Here is a list of all available formats in this data set.

In [25]:
sorted(T.formats)

['lex-orig-full', 'lex-trans-full', 'text-orig-full', 'text-trans-full']

## Using the formats
Now let's use those formats to print out the first verse of the Hebrew Bible.

In [26]:
for fmt in sorted(T.formats):
    print('{}:\n\t{}'.format(fmt, T.text(range(1,12), fmt=fmt)))

lex-orig-full:
	ܟܬܒܐ ܝܠܝܕܘܬܐ ܝܫܘܥ ܡܫܝܚܐ ܒܪܐ ܕܘܝܕ ܒܪܐ ܐܒܪܗܡ ܐܒܪܗܡ ܝܠܕ ܐܝܣܚܩ 
lex-trans-full:
	CTBA ;L;DOTA ;WOE MW;KA BRA DO;D BRA ABRHM ABRHM ;LD A;SKX 
text-orig-full:
	ܟܬܒܐ ܕܝܠܝܕܘܬܗ ܕܝܫܘܥ ܡܫܝܚܐ ܒܪܗ ܕܕܘܝܕ ܒܪܗ ܕܐܒܪܗܡ ܐܒܪܗܡ ܐܘܠܕ ܠܐܝܣܚܩ 
text-trans-full:
	CTBA D;L;DOTH D;WOE MW;KA BRH DDO;D BRH DABRHM ABRHM AOLD LA;SKX 


If we do not specify a format, the **default** format is used (`text-orig-full`).

In [27]:
print(T.text(range(1,12)))

ܟܬܒܐ ܕܝܠܝܕܘܬܗ ܕܝܫܘܥ ܡܫܝܚܐ ܒܪܗ ܕܕܘܝܕ ܒܪܗ ܕܐܒܪܗܡ ܐܒܪܗܡ ܐܘܠܕ ܠܐܝܣܚܩ 


## Whole text in all formats in less than a second
Part of the pleasure of working with computers is that they can crunch massive amounts of data.
The text of the Hebrew Bible is a piece of cake.

It takes just ten seconds to have that cake and eat it. 
In nearly a dozen formats.

In [28]:
indent(reset=True)
info('writing plain text of whole Syriac New Testament in all formats')

text = collections.defaultdict(list)

for v in F.otype.s('verse'):
    words = L.d(v, 'word')
    for fmt in sorted(T.formats):
        text[fmt].append(T.text(words, fmt=fmt))

info('done {} formats'.format(len(text)))

for fmt in sorted(text):
    print('{}\n{}\n'.format(fmt, '\n'.join(text[fmt][0:5])))

  0.00s writing plain text of whole Syriac New Testament in all formats
  0.85s done 4 formats
lex-orig-full
ܟܬܒܐ ܝܠܝܕܘܬܐ ܝܫܘܥ ܡܫܝܚܐ ܒܪܐ ܕܘܝܕ ܒܪܐ ܐܒܪܗܡ 
ܐܒܪܗܡ ܝܠܕ ܐܝܣܚܩ ܐܝܣܚܩ ܝܠܕ ܝܥܩܘܒ ܝܥܩܘܒ ܝܠܕ ܝܗܘܕܐ ܐܚܐ 
ܝܗܘܕܐ ܝܠܕ ܦܪܨ ܙܪܚ ܡܢ ܬܡܪ ܦܪܨ ܝܠܕ ܚܨܪܘܢ ܚܨܪܘܢ ܝܠܕ ܐܪܡ 
ܐܪܡ ܝܠܕ ܥܡܝܢܕܒ ܥܡܝܢܕܒ ܝܠܕ ܢܚܫܘܢ ܢܚܫܘܢ ܝܠܕ ܣܠܡܘܢ 
ܣܠܡܘܢ ܝܠܕ ܒܥܙ ܡܢ ܪܚܒ ܒܥܙ ܝܠܕ ܥܘܒܝܕ ܡܢ ܪܥܘܬ ܥܘܒܝܕ ܝܠܕ ܐܝܫܝ 

lex-trans-full
CTBA ;L;DOTA ;WOE MW;KA BRA DO;D BRA ABRHM 
ABRHM ;LD A;SKX A;SKX ;LD ;EXOB ;EXOB ;LD ;HODA AKA 
;HODA ;LD IR/ ZRK MN TMR IR/ ;LD K/RON K/RON ;LD ARM 
ARM ;LD EM;NDB EM;NDB ;LD NKWON NKWON ;LD SLMON 
SLMON ;LD BEZ MN RKB BEZ ;LD EOB;D MN REOT EOB;D ;LD A;W; 

text-orig-full
ܟܬܒܐ ܕܝܠܝܕܘܬܗ ܕܝܫܘܥ ܡܫܝܚܐ ܒܪܗ ܕܕܘܝܕ ܒܪܗ ܕܐܒܪܗܡ 
ܐܒܪܗܡ ܐܘܠܕ ܠܐܝܣܚܩ ܐܝܣܚܩ ܐܘܠܕ ܠܝܥܩܘܒ ܝܥܩܘܒ ܐܘܠܕ ܠܝܗܘܕܐ ܘܠܐܚܘܗܝ 
ܝܗܘܕܐ ܐܘܠܕ ܠܦܪܨ ܘܠܙܪܚ ܡܢ ܬܡܪ ܦܪܨ ܐܘܠܕ ܠܚܨܪܘܢ ܚܨܪܘܢ ܐܘܠܕ ܠܐܪܡ 
ܐܪܡ ܐܘܠܕ ܠܥܡܝܢܕܒ ܥܡܝܢܕܒ ܐܘܠܕ ܠܢܚܫܘܢ ܢܚܫܘܢ ܐܘܠܕ ܠܣܠܡܘܢ 
ܣܠܡܘܢ ܐܘܠܕ ܠܒܥܙ ܡܢ ܪܚܒ ܒܥܙ ܐܘܠܕ ܠܥܘܒܝܕ ܡܢ ܪܥܘܬ ܥܘܒܝܕ ܐܘܠܕ ܠܐܝܫܝ 

text-trans-full
CTBA D;L;DOTH D;WOE MW;KA BRH D

### The full plain text
We write a few formats to file, in your `Downloads` folder.

In [29]:
orig = 'text-orig-full'
trans = 'text-trans-full'
for fmt in (orig, trans):
    with open(os.path.expanduser(f'~/Downloads/{fmt}.txt'), 'w') as f:
        f.write('\n'.join(text[fmt]))

In [30]:
!head -n 20 ~/Downloads/{orig}.txt

ܟܬܒܐ ܕܝܠܝܕܘܬܗ ܕܝܫܘܥ ܡܫܝܚܐ ܒܪܗ ܕܕܘܝܕ ܒܪܗ ܕܐܒܪܗܡ 
ܐܒܪܗܡ ܐܘܠܕ ܠܐܝܣܚܩ ܐܝܣܚܩ ܐܘܠܕ ܠܝܥܩܘܒ ܝܥܩܘܒ ܐܘܠܕ ܠܝܗܘܕܐ ܘܠܐܚܘܗܝ 
ܝܗܘܕܐ ܐܘܠܕ ܠܦܪܨ ܘܠܙܪܚ ܡܢ ܬܡܪ ܦܪܨ ܐܘܠܕ ܠܚܨܪܘܢ ܚܨܪܘܢ ܐܘܠܕ ܠܐܪܡ 
ܐܪܡ ܐܘܠܕ ܠܥܡܝܢܕܒ ܥܡܝܢܕܒ ܐܘܠܕ ܠܢܚܫܘܢ ܢܚܫܘܢ ܐܘܠܕ ܠܣܠܡܘܢ 
ܣܠܡܘܢ ܐܘܠܕ ܠܒܥܙ ܡܢ ܪܚܒ ܒܥܙ ܐܘܠܕ ܠܥܘܒܝܕ ܡܢ ܪܥܘܬ ܥܘܒܝܕ ܐܘܠܕ ܠܐܝܫܝ 
ܐܝܫܝ ܐܘܠܕ ܠܕܘܝܕ ܡܠܟܐ ܕܘܝܕ ܐܘܠܕ ܠܫܠܝܡܘܢ ܡܢ ܐܢܬܬܗ ܕܐܘܪܝܐ 
ܫܠܝܡܘܢ ܐܘܠܕ ܠܪܚܒܥܡ ܪܚܒܥܡ ܐܘܠܕ ܠܐܒܝܐ ܐܒܝܐ ܐܘܠܕ ܠܐܣܐ 
ܐܣܐ ܐܘܠܕ ܠܝܗܘܫܦܛ ܝܗܘܫܦܛ ܐܘܠܕ ܠܝܘܪܡ ܝܘܪܡ ܐܘܠܕ ܠܥܘܙܝܐ 
ܥܘܙܝܐ ܐܘܠܕ ܠܝܘܬܡ ܝܘܬܡ ܐܘܠܕ ܠܐܚܙ ܐܚܙ ܐܘܠܕ ܠܚܙܩܝܐ 
ܚܙܩܝܐ ܐܘܠܕ ܠܡܢܫܐ ܡܢܫܐ ܐܘܠܕ ܠܐܡܘܢ ܐܡܘܢ ܐܘܠܕ ܠܝܘܫܝܐ 
ܝܘܫܝܐ ܐܘܠܕ ܠܝܘܟܢܝܐ ܘܠܐܚܘܗܝ ܒܓܠܘܬܐ ܕܒܒܠ 
ܡܢ ܒܬܪ ܓܠܘܬܐ ܕܝܢ ܕܒܒܠ ܝܘܟܢܝܐ ܐܘܠܕ ܠܫܠܬܐܝܠ ܫܠܬܐܝܠ ܐܘܠܕ ܠܙܘܪܒܒܠ 
ܙܘܪܒܒܠ ܐܘܠܕ ܠܐܒܝܘܕ ܐܒܝܘܕ ܐܘܠܕ ܠܐܠܝܩܝܡ ܐܠܝܩܝܡ ܐܘܠܕ ܠܥܙܘܪ 
ܥܙܘܪ ܐܘܠܕ ܠܙܕܘܩ ܙܕܘܩ ܐܘܠܕ ܠܐܟܝܢ ܐܟܝܢ ܐܘܠܕ ܠܐܠܝܘܕ 
ܐܠܝܘܕ ܐܘܠܕ ܠܐܠܝܥܙܪ ܐܠܝܥܙܪ ܐܘܠܕ ܠܡܬܢ ܡܬܢ ܐܘܠܕ ܠܝܥܩܘܒ 
ܝܥܩܘܒ ܐܘܠܕ ܠܝܘܣܦ ܓܒܪܗ ܕܡܪܝܡ ܕܡܢܗ ܐܬܝܠܕ ܝܫܘܥ ܕܡܬܩܪܐ ܡܫܝܚܐ 
ܟܠܗܝܢ ܗܟܝܠ ܫܪܒܬܐ ܡܢ ܐܒܪܗܡ ܥܕܡܐ ܠܕܘܝܕ ܫܪܒܬܐ ܐܪܒܥܣܪܐ ܘܡܢ ܕܘܝܕ ܥܕܡܐ ܠܓܠܘܬܐ ܕܒܒܠ ܫܪܒܬܐ ܐܪܒܥܣܪܐ ܘܡܢ ܓܠܘܬܐ ܕܒܒܠ ܥܕܡܐ ܠܡܫܝܚܐ ܫܪܒܬܐ ܐ

## Book names

For Bible book names, we can use several languages.
Well, in this case we have just English.

### Languages
Here are the languages that we can use for book names.
These languages come from the features `book@ll`, where `ll` is a two letter
ISO language code. Have a look in your data directory, you can't miss them.

In [31]:
T.languages

{'': {'language': 'default', 'languageEnglish': 'default'},
 'en': {'language': 'English', 'languageEnglish': 'English'}}

## Sections

A section is a book, a chapter or a verse.
Knowledge of sections is not baked into Text-Fabric. 
The config feature `otext.tf` may specify three section levels, and tell
what the corresponding node types and features are.

From that knowledge it can construct mappings from nodes to sections, e.g. from verse
nodes to tuples of the form:

    (bookName, chapterNumber, verseNumber)
   
Here are examples of getting the section that corresponds to a node and vice versa.

**NB:** `sectionFromNode` always delivers a verse specification, either from the
first slot belonging to that node, or, if `lastSlot`, from the last slot
belonging to that node.

In [32]:
for x in (
    ('section of first word',     T.sectionFromNode(1)                            ),
    ('node of Matthew 1:1',       T.nodeFromSection(('Matthew', 1, 1))            ),
    ('node of book Matthew',      T.nodeFromSection(('Matthew',))                 ),
    ('node of chapter Matthew 1', T.nodeFromSection(('Matthew', 1))               ),
    ('section of book node',      T.sectionFromNode(109641)                      ),
    ('idem, now last word',       T.sectionFromNode(109641, lastSlot=True)       ),
    ('section of chapter node',   T.sectionFromNode(109668)                      ),
    ('idem, now last word',       T.sectionFromNode(109668, lastSlot=True)       ),
): print('{:<30} {}'.format(*x))

section of first word          ('Matthew', 1, 1)
node of Matthew 1:1            112966
node of book Matthew           109641
node of chapter Matthew 1      109668
section of book node           ('Matthew', 1, 1)
idem, now last word            ('Matthew', 28, 20)
section of chapter node        ('Matthew', 1, 1)
idem, now last word            ('Matthew', 1, 25)
